In [1]:
import pandas as pd
import gc
import os
import glob
import ast
import numpy as np

# Load Company Matches CSV

In [2]:
df_company_matches = pd.read_csv('/home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/matching_results/company_matches_20250712_172703.csv')
df_people_matches = pd.read_csv('/home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/matching_results/people_matches_20250712_172703.csv')

In [3]:
df_company_matches = df_company_matches[['cb_uuid', 'cb_name', 'cb_harmonized', 'ps_person_id', 'ps_name', 'ps_harmonized', 'country_code']]

In [4]:
df_people_matches = df_people_matches[['cb_uuid', 'cb_name', 'cb_harmonized', 'ps_person_id', 'ps_name', 'ps_harmonized']] 

# Match with all Cleantech Patents

In [5]:
df_patstat_person_id = pd.read_csv('/mnt/hdd02/Projekt_EDV_TEK/edv_tek_all_cleantech_patstat_person_id.csv')
df_patstat_cleantech = pd.read_parquet('/mnt/hdd02/Projekt_EDV_TEK/edv_tek_cleantech_patents_title_abstract_date_cpc_ipc.parquet')

In [6]:
len(df_company_matches), len(df_people_matches), len(df_patstat_person_id), len(df_patstat_cleantech)

(5938627, 580172, 8086061, 1870863)

In [7]:
df_company_matches.head()

,cb_uuid,cb_name,cb_harmonized,ps_person_id,ps_name,ps_harmonized,country_code
0,21e77067-5537-408e-cad7-e5e72bb6ad86,Scribd,SCRIBD,11901439,"Scribd, Inc.",SCRIBD,USA
1,11352c70-21bb-b546-e9b2-723a4be03075,OpenX,OPENX,11521421,"OpenX Technologies, Inc.",OPENX,USA
2,11352c70-21bb-b546-e9b2-723a4be03075,OpenX,OPENX,11521421,"OpenX Technologies, Inc.",OPENX,USA
3,11352c70-21bb-b546-e9b2-723a4be03075,OpenX,OPENX,11521421,"OpenX Technologies, Inc.",OPENX,USA
4,11352c70-21bb-b546-e9b2-723a4be03075,OpenX,OPENX,11521421,"OpenX Technologies, Inc.",OPENX,USA


In [8]:
df_people_matches_patstat = pd.merge(df_people_matches, df_patstat_person_id, left_on='ps_person_id', right_on='person_id', how='inner')
df_people_matches_patstat.drop_duplicates(subset=['ps_person_id', 'appln_id'], inplace=True)
df_people_matches_patstat.to_csv('/home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/matching_cb_patents/people_matches_patstat.csv', index=False)

In [9]:
def process_company_matches_memory_efficient(df_company_matches, df_patstat_person_id, 
                                           chunk_size=50000, output_dir=None):
    """
    Process company matches in small chunks to avoid memory issues
    """
    if output_dir is None:
        output_dir = '/home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/matching_cb_patents/'
    
    os.makedirs(output_dir, exist_ok=True)
    
    total_matches = 0
    total_rows = len(df_company_matches)
    
    # Process in chunks
    for i, chunk_start in enumerate(range(0, total_rows, chunk_size)):
        chunk_end = min(chunk_start + chunk_size, total_rows)
        
        print(f"Processing chunk {i+1}: rows {chunk_start}-{chunk_end} "
              f"({chunk_end-chunk_start} rows, {(chunk_end/total_rows)*100:.1f}% complete)")
        
        # Get chunk - use .copy() to ensure we have a clean copy
        chunk = df_company_matches.iloc[chunk_start:chunk_end].copy()
        
        # Perform merge
        try:
            matches = pd.merge(chunk, df_patstat_person_id, 
                             left_on='ps_person_id', right_on='person_id', 
                             how='inner')
            
            # Remove duplicates
            matches.drop_duplicates(subset=['ps_person_id', 'appln_id'], inplace=True)
            
            if len(matches) > 0:
                # Save immediately
                output_file = f'{output_dir}company_matches_patstat_chunk_{i+1:03d}.csv'
                matches.to_csv(output_file, index=False)
                total_matches += len(matches)
                print(f"  → Found {len(matches)} matches, saved to chunk_{i+1:03d}.csv")
            else:
                print(f"  → No matches found")
                
        except Exception as e:
            print(f"  → Error processing chunk {i+1}: {e}")
            continue
        
        # Explicit cleanup
        del chunk, matches
        gc.collect()
    
    print(f"\nProcessing complete! Total matches found: {total_matches}")
    return total_matches

# Use the function
total_matches = process_company_matches_memory_efficient(
    df_company_matches, 
    df_patstat_person_id, 
    chunk_size=50000  # Start with 50k rows, adjust if needed
)

Processing chunk 1: rows 0-50000 (50000 rows, 0.8% complete)
  → Found 49471 matches, saved to chunk_001.csv
Processing chunk 2: rows 50000-100000 (50000 rows, 1.7% complete)
  → Found 49440 matches, saved to chunk_002.csv
Processing chunk 3: rows 100000-150000 (50000 rows, 2.5% complete)
  → Found 49328 matches, saved to chunk_003.csv
Processing chunk 4: rows 150000-200000 (50000 rows, 3.4% complete)
  → Found 47273 matches, saved to chunk_004.csv
Processing chunk 5: rows 200000-250000 (50000 rows, 4.2% complete)
  → Found 49662 matches, saved to chunk_005.csv
Processing chunk 6: rows 250000-300000 (50000 rows, 5.1% complete)
  → Found 51015 matches, saved to chunk_006.csv
Processing chunk 7: rows 300000-350000 (50000 rows, 5.9% complete)
  → Found 52395 matches, saved to chunk_007.csv
Processing chunk 8: rows 350000-400000 (50000 rows, 6.7% complete)
  → Found 43879 matches, saved to chunk_008.csv
Processing chunk 9: rows 400000-450000 (50000 rows, 7.6% complete)
  → Found 46565 matc

In [10]:
# Function to combine all chunk files into one final CSV file
def combine_chunk_files(output_dir):
    """
    Combine all chunk files into one final CSV file
    """
    chunk_files = glob.glob(f"{output_dir}company_matches_patstat_chunk_*.csv")
    
    if not chunk_files:
        print("No chunk files found to combine.")
        return
    
    combined_df = pd.concat((pd.read_csv(f) for f in chunk_files), ignore_index=True)
    combined_df.drop_duplicates(subset=['ps_person_id', 'appln_id'], inplace=True)
    
    final_output_file = f"{output_dir}company_matches_patstat_combined.csv"
    combined_df.to_csv(final_output_file, index=False)
    
    print(f"Combined {len(chunk_files)} chunk files into {final_output_file}")
    return combined_df

df_company_matches_patstat_combined = combine_chunk_files('/home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/matching_cb_patents/')

Combined 119 chunk files into /home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/matching_cb_patents/company_matches_patstat_combined.csv


In [11]:
df_company_matches_patstat_combined['appln_id'] = df_company_matches_patstat_combined['appln_id'].astype(str)
df_patstat_cleantech['appln_id'] = df_patstat_cleantech['appln_id'].astype(str)
df_company_matches_patstat_combined = df_company_matches_patstat_combined.merge(df_patstat_cleantech[['appln_id', 'ipc_class_symbol', 'cpc_class_symbol']], on='appln_id', how='left')

In [12]:
df_people_matches_patstat['appln_id'] = df_people_matches_patstat['appln_id'].astype(str)
df_person_matches_patstat_combined = pd.merge(df_people_matches_patstat, df_patstat_cleantech[['appln_id', 'ipc_class_symbol', 'cpc_class_symbol']], on='appln_id', how='left')

In [13]:
len(df_company_matches_patstat_combined), len(df_person_matches_patstat_combined)

(1162917, 400778)

In [14]:
df_person_matches_patstat_combined.head()

,cb_uuid,cb_name,cb_harmonized,ps_person_id,ps_name,ps_harmonized,appln_id,person_id,ipc_class_symbol,cpc_class_symbol
0,5ceca97b-493c-1446-6249-5aaa33464763,Kevin Flaherty,KEVIN FLAHERTY,45307436,"Flaherty, Kevin",FLAHERTY KEVIN,411568025,45307436,"[H01L 31/042, H01L 31/052, H01L 31/18]","[Y02E 10/47, Y02E 10/50]"
1,3b598c59-7b6c-2d48-763c-da55bca77035,Owen Byrne,OWEN BY,48467086,"Owen, Byron G.",OWEN BYRON,448273380,48467086,[E21B 43/16],[None]
2,a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,MARK ZUCKERBERG,11063428,"Zuckerberg, Mark E.",ZUCKERBERG MARK,364671708,11063428,[G06Q 50/00],[None]
3,a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,MARK ZUCKERBERG,11063428,"Zuckerberg, Mark E.",ZUCKERBERG MARK,331523353,11063428,"[G06Q 30/00, G06Q 30/02]",[None]
4,a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,MARK ZUCKERBERG,11063428,"Zuckerberg, Mark E.",ZUCKERBERG MARK,421488729,11063428,[G06Q 50/00],[None]


# Match with Crunchbase Performance Data

In [15]:
df_cb_funding_rounds = pd.read_csv('/mnt/hdd02/Projekt_EDV_TEK/CrunchBase Dump TUHH/funding_rounds.csv')

In [16]:
df_cb_funding_rounds.columns

Index(['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'country_code', 'state_code', 'region', 'city',
       'investment_type', 'announced_on', 'raised_amount_usd', 'raised_amount',
       'raised_amount_currency_code', 'post_money_valuation_usd',
       'post_money_valuation', 'post_money_valuation_currency_code',
       'investor_count', 'org_uuid', 'org_name', 'lead_investor_uuids'],
      dtype='object')

In [17]:
df_cb_funding_rounds.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,...,raised_amount_usd,raised_amount,raised_amount_currency_code,post_money_valuation_usd,post_money_valuation,post_money_valuation_currency_code,investor_count,org_uuid,org_name,lead_investor_uuids
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Meta,funding_round,facebook-angel--8a945939,https://www.crunchbase.com/funding_round/faceb...,393543.0,2007-05-27 06:08:18,2018-02-12 23:05:39,USA,CA,...,500000.0,500000.0,USD,NaN,NaN,NaN,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Meta,funding_round,facebook-series-a--d950d7a5,https://www.crunchbase.com/funding_round/faceb...,307991.0,2007-05-27 06:09:10,2018-02-12 23:52:16,USA,CA,...,12700000.0,12700000.0,USD,98000000.0,98000000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,b08efc27-da40-505a-6f9d-c9e14247bf36
2,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Meta,funding_round,facebook-series-b--6fae3958,https://www.crunchbase.com/funding_round/faceb...,108049.0,2007-05-27 06:09:36,2018-02-12 23:30:46,USA,CA,...,27500000.0,27500000.0,USD,502500000.0,502500000.0,USD,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-..."
3,bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,funding_round,photobucket-series-b--bcd5a63d,https://www.crunchbase.com/funding_round/photo...,469186.0,2007-05-29 11:05:59,2018-02-12 23:27:36,USA,CO,...,10500000.0,10500000.0,USD,NaN,NaN,NaN,2.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,Photobucket,NaN
4,60e6afd9-1215-465a-dd17-0ed600d4e29b,Series A - Geni,funding_round,geni-series-a--60e6afd9,https://www.crunchbase.com/funding_round/geni-...,497495.0,2007-05-31 20:19:28,2018-02-12 23:41:29,USA,CA,...,NaN,NaN,NaN,10000000.0,10000000.0,USD,1.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,fb2f8884-ec07-895a-48d7-d9a9d4d7175c


# Match Patent Data with Funding Rounds

In [18]:
# Match company patents with funding rounds (one-to-many)
print("Matching company patents with funding rounds...")
print(f"Company patents before matching: {len(df_company_matches_patstat_combined)}")
print(f"Funding rounds available: {len(df_cb_funding_rounds)}")

# Perform the merge - this will create multiple rows for companies with multiple funding rounds
df_company_patents_funding = pd.merge(
    df_company_matches_patstat_combined, 
    df_cb_funding_rounds, 
    left_on='cb_uuid', 
    right_on='org_uuid',
    how='inner'
)

print(f"Company patents after matching: {len(df_company_patents_funding)}")
print(f"Unique companies with funding data: {df_company_patents_funding['cb_uuid'].nunique()}")

# Check how many funding rounds per company
funding_per_company = df_company_patents_funding.groupby('cb_uuid')['uuid'].nunique().describe()
print(f"\nFunding rounds per company statistics:")
print(funding_per_company)

Matching company patents with funding rounds...
Company patents before matching: 1162917
Funding rounds available: 545920
Company patents after matching: 467532
Unique companies with funding data: 17608

Funding rounds per company statistics:
count    17608.000000
mean         2.880850
std          2.649838
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         33.000000
Name: uuid, dtype: float64


In [19]:
# Explore the matched data
print("=== COMPANY PATENTS WITH FUNDING DATA ===")
print(f"Total records: {len(df_company_patents_funding)}")
print(f"Records with funding data: {len(df_company_patents_funding[df_company_patents_funding['uuid'].notna()])}")

# Show sample of matched data
print("\nSample of company patents with funding:")
sample_with_funding = df_company_patents_funding[df_company_patents_funding['uuid'].notna()].head()
print(sample_with_funding[['cb_name', 'ps_name', 'appln_id', 'type', 'raised_amount_usd', 'created_at']].to_string())

=== COMPANY PATENTS WITH FUNDING DATA ===
Total records: 467532
Records with funding data: 467532

Sample of company patents with funding:
  cb_name       ps_name   appln_id           type  raised_amount_usd           created_at
0  Scribd  Scribd, Inc.  577867352  funding_round            40000.0  2007-06-02 07:36:21
1  Scribd  Scribd, Inc.  577867352  funding_round          3710000.0  2007-06-02 07:49:08
2  Scribd  Scribd, Inc.  577867352  funding_round          9000000.0  2008-12-19 09:01:24
3  Scribd  Scribd, Inc.  577867352  funding_round         13000000.0  2011-01-20 05:02:12
4  Scribd  Scribd, Inc.  577867352  funding_round         22000000.0  2015-01-05 09:31:05


In [20]:
# Save the matched results
output_dir = '/home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/final_results/'
os.makedirs(output_dir, exist_ok=True)

# Save company patents with funding data
df_company_patents_funding.to_csv(f'{output_dir}company_patents_funding_matched.csv', index=False)
print(f"Saved company patents with funding data: {len(df_company_patents_funding)} records")

# Create summary statistics
print("\n=== SUMMARY STATISTICS ===")
print(f"Total unique companies with patents: {df_company_patents_funding['cb_uuid'].nunique()}")
print(f"Total unique patents (company): {df_company_patents_funding['appln_id'].nunique()}")

# Funding statistics
company_funding_stats = df_company_patents_funding[df_company_patents_funding['raised_amount_usd'].notna()]['raised_amount_usd']

print(f"\nFunding amount statistics (Company patents):")
print(f"  Total funding rounds: {len(company_funding_stats)}")
print(f"  Total amount raised: ${company_funding_stats.sum():,.2f}")
print(f"  Average funding per round: ${company_funding_stats.mean():,.2f}")
print(f"  Median funding per round: ${company_funding_stats.median():,.2f}")

Saved company patents with funding data: 467532 records

=== SUMMARY STATISTICS ===
Total unique companies with patents: 17608
Total unique patents (company): 152084

Funding amount statistics (Company patents):
  Total funding rounds: 377773
  Total amount raised: $220,835,448,085,589.00
  Average funding per round: $584,571,814.52
  Median funding per round: $9,000,000.00


In [21]:
# Analyze funding types and patterns
print("=== FUNDING TYPE ANALYSIS ===")

# Company funding types
print("Company Patents - Funding Types:")
company_funding_types = df_company_patents_funding[df_company_patents_funding['type'].notna()]['type'].value_counts()
print(company_funding_types.head(10))

# Temporal analysis - extract year from created_at
df_company_patents_funding['funding_year'] = pd.to_datetime(df_company_patents_funding['created_at']).dt.year

print("\n=== TEMPORAL FUNDING ANALYSIS ===")
print("Company Patents - Funding by Year (top 10):")
company_yearly = df_company_patents_funding[df_company_patents_funding['funding_year'].notna()]['funding_year'].value_counts().head(10)
print(company_yearly)

=== FUNDING TYPE ANALYSIS ===
Company Patents - Funding Types:
type
funding_round    467532
Name: count, dtype: int64

=== TEMPORAL FUNDING ANALYSIS ===
Company Patents - Funding by Year (top 10):
funding_year
2021    52484
2017    48753
2022    41329
2014    37338
2020    37080
2015    36359
2013    33521
2016    33469
2018    33382
2019    29071
Name: count, dtype: int64


# TEK Analysis

In [22]:
# CPC class symbols sind als Listen gespeichert - first convert from string to actual lists
def safe_eval_list(x):
    try:
        if pd.isna(x) or x == '[None]' or x == 'None':
            return []
        if isinstance(x, str):
            # Remove [None] entries and convert to list
            cleaned = x.replace('[None]', '').replace('None', '').strip('[]')
            if not cleaned:
                return []
            # Split by comma and clean up
            items = [item.strip().strip("'\"") for item in cleaned.split(',') if item.strip() and item.strip() != 'None']
            return items
        if isinstance(x, np.ndarray):
            return [item for item in x if item and item != 'None']
        return []
    except:
        return []

# Extract Y02 classes from CPC class symbols
df_company_patents_funding['cpc_classes'] = df_company_patents_funding['cpc_class_symbol'].apply(safe_eval_list)

# Extract Y02 codes specifically
def extract_y02_codes(cpc_list):
    if not cpc_list:
        return []
    y02_codes = []
    for code in cpc_list:
        if isinstance(code, str) and code.startswith('Y02'):
            # Extract Y02 + next letter (e.g., Y02E, Y02T, etc.)
            y02_main = code[:4]  # Y02E, Y02T, etc.
            y02_codes.append(y02_main)
    return list(set(y02_codes))  # Remove duplicates

df_company_patents_funding['y02_classes'] = df_company_patents_funding['cpc_classes'].apply(extract_y02_codes)

# Filter only companies with Y02 codes (TEKs)
df_tek_funding = df_company_patents_funding[df_company_patents_funding['y02_classes'].apply(len) > 0].copy()

print(f"TEK companies with Y02 codes: {len(df_tek_funding)}")
print(f"Unique Y02 classes found: {set([code for codes in df_tek_funding['y02_classes'] for code in codes])}")

TEK companies with Y02 codes: 122456
Unique Y02 classes found: {'Y02T', 'Y02E', 'Y02C', 'Y02W', 'Y02B', 'Y02A', 'Y02P', 'Y02D'}


In [23]:
# Explode Y02 classes so each row represents one company-patent-Y02class combination
df_tek_exploded = df_tek_funding.explode('y02_classes').copy()
df_tek_exploded = df_tek_exploded[df_tek_exploded['y02_classes'].notna()]

print(f"Exploded TEK data: {len(df_tek_exploded)} records")
print("Y02 class distribution:")
print(df_tek_exploded['y02_classes'].value_counts())

Exploded TEK data: 122456 records
Y02 class distribution:
y02_classes
Y02E    40859
Y02A    32425
Y02D    15863
Y02P    14316
Y02B    10747
Y02T     6432
Y02W     1123
Y02C      691
Name: count, dtype: int64


In [24]:
# Explode Y02 classes so each row represents one company-patent-Y02class combination
df_tek_exploded = df_tek_funding.explode('y02_classes').copy()
df_tek_exploded = df_tek_exploded[df_tek_exploded['y02_classes'].notna()]

print(f"Exploded TEK data: {len(df_tek_exploded)} records")
print("Y02 class distribution:")
print(df_tek_exploded['y02_classes'].value_counts())

Exploded TEK data: 122456 records
Y02 class distribution:
y02_classes
Y02E    40859
Y02A    32425
Y02D    15863
Y02P    14316
Y02B    10747
Y02T     6432
Y02W     1123
Y02C      691
Name: count, dtype: int64


# Patent Novelty Analysis

Now we'll analyze the same patent-funding matching data but categorize patents by their novelty scores (High, Medium, Low novelty) to understand how patent novelty relates to funding patterns.

In [25]:
# Check if novelty data is available (from your AP 2)
novelty_file_path = '/home/thiesen/Documents/Projekt_EDV-TEK/AP 3 - Diffusion von TEKs/edv_tek_diffusion_patent_novelty_scores.csv'

try:
    df_novelty = pd.read_csv(novelty_file_path)
    print(f"Novelty data loaded: {len(df_novelty)} patents")
    
    # Create novelty categories
    df_novelty['novelty_q100_percentile'] = df_novelty['novelty_q100'].rank(pct=True) * 100
    df_novelty['novelty_category'] = 'Medium'
    df_novelty.loc[df_novelty['novelty_q100_percentile'] >= 90, 'novelty_category'] = 'High'
    df_novelty.loc[df_novelty['novelty_q100_percentile'] <= 10, 'novelty_category'] = 'Low'
    
    # Convert appln_id to string for merging
    df_novelty['appln_id'] = df_novelty['appln_id'].astype(str)
    df_tek_exploded['appln_id'] = df_tek_exploded['appln_id'].astype(str)
    
    # Merge with TEK funding data
    df_tek_funding_novelty = pd.merge(
        df_tek_exploded,
        df_novelty[['appln_id', 'novelty_q100', 'novelty_category']],
        on='appln_id',
        how='left'
    )
    
    print(f"TEK funding with novelty: {len(df_tek_funding_novelty)} records")
    print("Novelty category distribution:")
    print(df_tek_funding_novelty['novelty_category'].value_counts())
    
except FileNotFoundError:
    print("Novelty data not found - will proceed without novelty analysis")
    df_tek_funding_novelty = df_tek_exploded.copy()
    df_tek_funding_novelty['novelty_category'] = 'Unknown'

Novelty data loaded: 318104 patents
TEK funding with novelty: 122456 records
Novelty category distribution:
novelty_category
Medium    35865
High      12080
Low        2097
Name: count, dtype: int64


In [26]:
# Categorize patents by novelty levels (following the same logic as the GPU script)
print("Categorizing patents by novelty levels...")

# Calculate percentiles for novelty_q100 among all patents with novelty scores
df_novelty['novelty_q100_percentile'] = df_novelty['novelty_q100'].rank(pct=True, method='average') * 100

# Classify patents into novelty categories based on percentiles
df_novelty['novelty_category'] = 'Medium'  # Default
df_novelty.loc[df_novelty['novelty_q100_percentile'] >= 90, 'novelty_category'] = 'High'
df_novelty.loc[df_novelty['novelty_q100_percentile'] <= 10, 'novelty_category'] = 'Low'

print(f"Novelty categories distribution:")
novelty_counts = df_novelty['novelty_category'].value_counts()
print(novelty_counts)
print(f"Percentages: {(novelty_counts / len(df_novelty) * 100).round(1).to_dict()}")

# Convert appln_id to string for consistent merging
df_novelty['appln_id'] = df_novelty['appln_id'].astype(str)

Categorizing patents by novelty levels...
Novelty categories distribution:
novelty_category
Medium    254491
Low        31810
High       31803
Name: count, dtype: int64
Percentages: {'Medium': 80.0, 'Low': 10.0, 'High': 10.0}


In [27]:
# Merge novelty data with the existing company patent funding data
print("Merging novelty data with company patent funding data...")
print(f"Company patents funding before novelty merge: {len(df_company_patents_funding)}")
print(f"Unique patents in company funding data: {df_company_patents_funding['appln_id'].nunique()}")

# Check overlap between datasets
company_patent_ids = set(df_company_patents_funding['appln_id'].astype(str))
novelty_patent_ids = set(df_novelty['appln_id'])
overlap_ids = company_patent_ids.intersection(novelty_patent_ids)

print(f"Patents in company funding data: {len(company_patent_ids)}")
print(f"Patents with novelty scores: {len(novelty_patent_ids)}")
print(f"Overlapping patents: {len(overlap_ids)}")
print(f"Overlap percentage: {len(overlap_ids)/len(company_patent_ids)*100:.1f}%")

# Merge with novelty data (inner join to keep only patents with novelty scores)
df_company_patents_funding_novelty = pd.merge(
    df_company_patents_funding,
    df_novelty[['appln_id', 'novelty_q100', 'novelty_q90', 'novelty_q50', 'novelty_q100_percentile', 'novelty_category']],
    on='appln_id',
    how='inner'  # Only keep patents that have novelty scores
)

print(f"Company patents funding after novelty merge: {len(df_company_patents_funding_novelty)}")
print(f"Unique patents with novelty scores: {df_company_patents_funding_novelty['appln_id'].nunique()}")
print(f"Unique companies with novelty-scored patents: {df_company_patents_funding_novelty['cb_uuid'].nunique()}")

Merging novelty data with company patent funding data...
Company patents funding before novelty merge: 467532
Unique patents in company funding data: 152084
Patents in company funding data: 152084
Patents with novelty scores: 318104
Overlapping patents: 39311
Overlap percentage: 25.8%
Company patents funding after novelty merge: 156355
Unique patents with novelty scores: 39311
Unique companies with novelty-scored patents: 6563


In [28]:
# Analyze funding patterns by novelty categories
print("=== FUNDING ANALYSIS BY NOVELTY CATEGORIES ===")

# Distribution of patents by novelty category
print("Patent distribution by novelty category:")
novelty_distribution = df_company_patents_funding_novelty['novelty_category'].value_counts()
print(novelty_distribution)
print(f"Percentages: {(novelty_distribution / len(df_company_patents_funding_novelty) * 100).round(1).to_dict()}")

# Funding statistics by novelty category
print("\n=== FUNDING STATISTICS BY NOVELTY CATEGORY ===")
for category in ['High', 'Medium', 'Low']:
    category_data = df_company_patents_funding_novelty[
        (df_company_patents_funding_novelty['novelty_category'] == category) &
        (df_company_patents_funding_novelty['raised_amount_usd'].notna())
    ]
    
    if len(category_data) > 0:
        print(f"\n{category} Novelty Patents:")
        print(f"  Total funding rounds: {len(category_data)}")
        print(f"  Unique companies: {category_data['cb_uuid'].nunique()}")
        print(f"  Unique patents: {category_data['appln_id'].nunique()}")
        print(f"  Total amount raised: ${category_data['raised_amount_usd'].sum():,.2f}")
        print(f"  Average funding per round: ${category_data['raised_amount_usd'].mean():,.2f}")
        print(f"  Median funding per round: ${category_data['raised_amount_usd'].median():,.2f}")
    else:
        print(f"\n{category} Novelty Patents: No funding data available")

=== FUNDING ANALYSIS BY NOVELTY CATEGORIES ===
Patent distribution by novelty category:
novelty_category
Medium    118584
High       29684
Low         8087
Name: count, dtype: int64
Percentages: {'Medium': 75.8, 'High': 19.0, 'Low': 5.2}

=== FUNDING STATISTICS BY NOVELTY CATEGORY ===

High Novelty Patents:
  Total funding rounds: 25670
  Unique companies: 1081
  Unique patents: 5466
  Total amount raised: $2,006,587,209,891.00
  Average funding per round: $78,168,570.70
  Median funding per round: $11,000,000.00

Medium Novelty Patents:
  Total funding rounds: 100023
  Unique companies: 4999
  Unique patents: 25780
  Total amount raised: $40,049,765,411,050.00
  Average funding per round: $400,405,560.83
  Median funding per round: $8,713,627.00

Low Novelty Patents:
  Total funding rounds: 6789
  Unique companies: 848
  Unique patents: 2017
  Total amount raised: $3,567,592,454,791.00
  Average funding per round: $525,496,016.32
  Median funding per round: $8,500,000.00


In [29]:
# Analyze funding types by novelty category
print("=== FUNDING TYPE ANALYSIS BY NOVELTY CATEGORY ===")

for category in ['High', 'Medium', 'Low']:
    category_data = df_company_patents_funding_novelty[
        (df_company_patents_funding_novelty['novelty_category'] == category) &
        (df_company_patents_funding_novelty['type'].notna())
    ]
    
    if len(category_data) > 0:
        print(f"\n{category} Novelty Patents - Top 5 Funding Types:")
        funding_types = category_data['type'].value_counts().head(5)
        for fund_type, count in funding_types.items():
            percentage = (count / len(category_data)) * 100
            print(f"  {fund_type}: {count} ({percentage:.1f}%)")

# Temporal analysis by novelty category
print("\n=== TEMPORAL FUNDING ANALYSIS BY NOVELTY CATEGORY ===")
df_company_patents_funding_novelty['funding_year'] = pd.to_datetime(df_company_patents_funding_novelty['created_at']).dt.year

for category in ['High', 'Medium', 'Low']:
    category_data = df_company_patents_funding_novelty[
        (df_company_patents_funding_novelty['novelty_category'] == category) &
        (df_company_patents_funding_novelty['funding_year'].notna())
    ]
    
    if len(category_data) > 0:
        print(f"\n{category} Novelty Patents - Top 5 Funding Years:")
        yearly_funding = category_data['funding_year'].value_counts().head(5)
        for year, count in yearly_funding.items():
            print(f"  {int(year)}: {count} funding rounds")

=== FUNDING TYPE ANALYSIS BY NOVELTY CATEGORY ===

High Novelty Patents - Top 5 Funding Types:
  funding_round: 29684 (100.0%)

Medium Novelty Patents - Top 5 Funding Types:
  funding_round: 118584 (100.0%)

Low Novelty Patents - Top 5 Funding Types:
  funding_round: 8087 (100.0%)

=== TEMPORAL FUNDING ANALYSIS BY NOVELTY CATEGORY ===

High Novelty Patents - Top 5 Funding Years:
  2021: 4580 funding rounds
  2022: 2812 funding rounds
  2020: 2749 funding rounds
  2017: 2678 funding rounds
  2013: 2668 funding rounds

Medium Novelty Patents - Top 5 Funding Years:
  2021: 14539 funding rounds
  2017: 11298 funding rounds
  2022: 10615 funding rounds
  2013: 9949 funding rounds
  2015: 9439 funding rounds

Low Novelty Patents - Top 5 Funding Years:
  2017: 919 funding rounds
  2021: 915 funding rounds
  2022: 725 funding rounds
  2015: 642 funding rounds
  2014: 628 funding rounds


In [30]:
# Comparative analysis across novelty categories
print("=== COMPARATIVE FUNDING ANALYSIS ===")

# Create summary table
summary_data = []
for category in ['High', 'Medium', 'Low']:
    category_data = df_company_patents_funding_novelty[
        df_company_patents_funding_novelty['novelty_category'] == category
    ]
    
    category_funding = category_data[category_data['raised_amount_usd'].notna()]
    
    summary_data.append({
        'Novelty_Category': category,
        'Total_Patents': len(category_data),
        'Unique_Companies': category_data['cb_uuid'].nunique(),
        'Unique_Patents': category_data['appln_id'].nunique(),
        'Funding_Rounds': len(category_funding),
        'Total_Funding_USD': category_funding['raised_amount_usd'].sum() if len(category_funding) > 0 else 0,
        'Avg_Funding_USD': category_funding['raised_amount_usd'].mean() if len(category_funding) > 0 else 0,
        'Median_Funding_USD': category_funding['raised_amount_usd'].median() if len(category_funding) > 0 else 0
    })

df_novelty_summary = pd.DataFrame(summary_data)
print("\nSummary by Novelty Category:")
print(df_novelty_summary.to_string(index=False, float_format='%.2f'))

# Calculate funding success rates (companies with funding vs total companies)
print("\n=== FUNDING SUCCESS RATES BY NOVELTY ===")
for category in ['High', 'Medium', 'Low']:
    category_data = df_company_patents_funding_novelty[
        df_company_patents_funding_novelty['novelty_category'] == category
    ]
    
    total_companies = category_data['cb_uuid'].nunique()
    funded_companies = category_data[category_data['raised_amount_usd'].notna()]['cb_uuid'].nunique()
    
    if total_companies > 0:
        success_rate = (funded_companies / total_companies) * 100
        print(f"{category} Novelty: {funded_companies}/{total_companies} companies funded ({success_rate:.1f}%)")
    else:
        print(f"{category} Novelty: No companies found")

=== COMPARATIVE FUNDING ANALYSIS ===

Summary by Novelty Category:
Novelty_Category  Total_Patents  Unique_Companies  Unique_Patents  Funding_Rounds  Total_Funding_USD  Avg_Funding_USD  Median_Funding_USD
            High          29684              1235            6250           25670   2006587209891.00      78168570.70         11000000.00
          Medium         118584              5820           30626          100023  40049765411050.00     400405560.83          8713627.00
             Low           8087               998            2435            6789   3567592454791.00     525496016.32          8500000.00

=== FUNDING SUCCESS RATES BY NOVELTY ===
High Novelty: 1081/1235 companies funded (87.5%)
Medium Novelty: 4999/5820 companies funded (85.9%)
Low Novelty: 848/998 companies funded (85.0%)


In [31]:
# Statistical analysis of novelty vs funding relationships
print("=== STATISTICAL ANALYSIS: NOVELTY vs FUNDING ===")

# Correlation between novelty scores and funding amounts
funded_patents = df_company_patents_funding_novelty[
    df_company_patents_funding_novelty['raised_amount_usd'].notna()
].copy()

if len(funded_patents) > 0:
    print("Correlation between novelty scores and funding amounts:")
    
    # Calculate correlations
    correlations = {}
    for novelty_col in ['novelty_q100', 'novelty_q90', 'novelty_q50']:
        corr = funded_patents[novelty_col].corr(funded_patents['raised_amount_usd'])
        correlations[novelty_col] = corr
        print(f"  {novelty_col} vs funding amount: {corr:.4f}")
    
    # Test for significant differences in funding amounts between novelty categories
    from scipy.stats import kruskal
    
    high_funding = funded_patents[funded_patents['novelty_category'] == 'High']['raised_amount_usd']
    medium_funding = funded_patents[funded_patents['novelty_category'] == 'Medium']['raised_amount_usd']
    low_funding = funded_patents[funded_patents['novelty_category'] == 'Low']['raised_amount_usd']
    
    # Remove empty groups for statistical testing
    groups = [group for group in [high_funding, medium_funding, low_funding] if len(group) > 0]
    group_names = [name for name, group in zip(['High', 'Medium', 'Low'], [high_funding, medium_funding, low_funding]) if len(group) > 0]
    
    if len(groups) >= 2:
        statistic, p_value = kruskal(*groups)
        print(f"\nKruskal-Wallis test for funding differences between novelty categories:")
        print(f"  Test statistic: {statistic:.4f}")
        print(f"  P-value: {p_value:.6f}")
        print(f"  Groups tested: {group_names}")
        
        if p_value < 0.05:
            print("  → Significant differences in funding amounts between novelty categories")
        else:
            print("  → No significant differences in funding amounts between novelty categories")
    else:
        print("\nInsufficient data for statistical testing")
else:
    print("No funded patents available for statistical analysis")

=== STATISTICAL ANALYSIS: NOVELTY vs FUNDING ===
Correlation between novelty scores and funding amounts:
  novelty_q100 vs funding amount: -0.0354
  novelty_q90 vs funding amount: -0.0245
  novelty_q50 vs funding amount: -0.0081

Kruskal-Wallis test for funding differences between novelty categories:
  Test statistic: 471.2584
  P-value: 0.000000
  Groups tested: ['High', 'Medium', 'Low']
  → Significant differences in funding amounts between novelty categories


In [32]:
# Save the novelty-enhanced results
print("=== SAVING NOVELTY-ENHANCED RESULTS ===")

# Save the complete dataset with novelty information
novelty_output_file = f'{output_dir}company_patents_funding_with_novelty.csv'
df_company_patents_funding_novelty.to_csv(novelty_output_file, index=False)
print(f"Saved company patents with funding and novelty data: {len(df_company_patents_funding_novelty)} records")

# Save the summary statistics
novelty_summary_file = f'{output_dir}novelty_funding_summary.csv'
df_novelty_summary.to_csv(novelty_summary_file, index=False)
print(f"Saved novelty funding summary to: {novelty_summary_file}")

# Create detailed breakdown by novelty category
print("\n=== DETAILED BREAKDOWN BY NOVELTY CATEGORY ===")

for category in ['High', 'Medium', 'Low']:
    category_data = df_company_patents_funding_novelty[
        df_company_patents_funding_novelty['novelty_category'] == category
    ]
    
    if len(category_data) > 0:
        # Save category-specific data
        category_file = f'{output_dir}company_patents_funding_{category.lower()}_novelty.csv'
        category_data.to_csv(category_file, index=False)
        
        print(f"\n{category} Novelty Patents:")
        print(f"  File saved: {category_file}")
        print(f"  Records: {len(category_data)}")
        print(f"  Unique companies: {category_data['cb_uuid'].nunique()}")
        print(f"  Unique patents: {category_data['appln_id'].nunique()}")
        
        # Funding summary for this category
        category_funding = category_data[category_data['raised_amount_usd'].notna()]
        if len(category_funding) > 0:
            print(f"  Total funding: ${category_funding['raised_amount_usd'].sum():,.2f}")
            print(f"  Companies with funding: {category_funding['cb_uuid'].nunique()}")

print(f"\nAll novelty-enhanced files saved to: {output_dir}")

# Final summary
print("\n" + "="*60)
print("FINAL SUMMARY: PATENT NOVELTY AND FUNDING ANALYSIS")
print("="*60)
print(f"Total patents with funding and novelty data: {len(df_company_patents_funding_novelty):,}")
print(f"Original patents with funding data: {len(df_company_patents_funding):,}")
print(f"Percentage with novelty scores: {len(df_company_patents_funding_novelty)/len(df_company_patents_funding)*100:.1f}%")
print(f"High novelty patents: {len(df_company_patents_funding_novelty[df_company_patents_funding_novelty['novelty_category'] == 'High']):,}")
print(f"Medium novelty patents: {len(df_company_patents_funding_novelty[df_company_patents_funding_novelty['novelty_category'] == 'Medium']):,}")
print(f"Low novelty patents: {len(df_company_patents_funding_novelty[df_company_patents_funding_novelty['novelty_category'] == 'Low']):,}")
print("="*60)

=== SAVING NOVELTY-ENHANCED RESULTS ===
Saved company patents with funding and novelty data: 156355 records
Saved novelty funding summary to: /home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/final_results/novelty_funding_summary.csv

=== DETAILED BREAKDOWN BY NOVELTY CATEGORY ===

High Novelty Patents:
  File saved: /home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/final_results/company_patents_funding_high_novelty.csv
  Records: 29684
  Unique companies: 1235
  Unique patents: 6250
  Total funding: $2,006,587,209,891.00
  Companies with funding: 1081

Medium Novelty Patents:
  File saved: /home/thiesen/Documents/Projekt_EDV-TEK/AP 4 - Kommerzialisierung von TEKs/Matching Crunchbase and PATSTAT Data/final_results/company_patents_funding_medium_novelty.csv
  Records: 118584
  Unique companies: 5820
  Unique patents: 30626
  Total funding: $40,049,765,411,050.00
  

# Temporal Data Analysis

In [33]:
# Create temporal variables
df_tek_funding_novelty['funding_date'] = pd.to_datetime(df_tek_funding_novelty['announced_on'])
df_tek_funding_novelty['funding_year'] = df_tek_funding_novelty['funding_date'].dt.year
df_tek_funding_novelty['funding_month'] = df_tek_funding_novelty['funding_date'].dt.month

# Filter realistic years
df_tek_funding_novelty = df_tek_funding_novelty[
    (df_tek_funding_novelty['funding_year'] >= 2000) & 
    (df_tek_funding_novelty['funding_year'] <= 2024)
]

print("Funding year distribution:")
print(df_tek_funding_novelty['funding_year'].value_counts().sort_index().tail(10))

Funding year distribution:
funding_year
2014    9471
2015    7310
2016    7435
2017    6395
2018    7094
2019    6464
2020    5579
2021    6453
2022    7691
2023    4399
Name: count, dtype: int64


In [34]:
# Y02-specific commercialization analysis
y02_analysis = df_tek_funding_novelty.groupby('y02_classes').agg({
    'cb_uuid': 'nunique',  # Unique companies
    'appln_id': 'nunique',  # Unique patents
    'raised_amount_usd': ['sum', 'mean', 'count'],
    'post_money_valuation_usd': 'mean',
    'investor_count': 'mean'
}).round(2)

print("TEK Categories Commercialization Analysis:")
print(y02_analysis)

TEK Categories Commercialization Analysis:
            cb_uuid appln_id raised_amount_usd                       \
            nunique  nunique               sum          mean  count   
y02_classes                                                           
Y02A           1571     6856      1.126900e+12  3.888276e+07  28982   
Y02B            755     3415      3.246363e+12  3.903286e+08   8317   
Y02C             73      181      1.411143e+10  2.519898e+07    560   
Y02D            946     5527      7.928480e+13  6.301447e+09  12582   
Y02E           1838    12501      2.813509e+12  8.295765e+07  33915   
Y02P           1236     4258      1.373439e+12  1.150573e+08  11937   
Y02T            562     2228      3.682841e+11  7.228343e+07   5095   
Y02W            202      410      3.904737e+10  4.437201e+07    880   

            post_money_valuation_usd investor_count  
                                mean           mean  
y02_classes                                          
Y02A         

In [35]:
# Y02-specific commercialization analysis
y02_analysis = df_tek_funding_novelty.groupby('y02_classes').agg({
    'cb_uuid': 'nunique',  # Unique companies
    'appln_id': 'nunique',  # Unique patents
    'raised_amount_usd': ['sum', 'mean', 'count'],
    'post_money_valuation_usd': 'mean',
    'investor_count': 'mean'
}).round(2)

print("TEK Categories Commercialization Analysis:")
print(y02_analysis)

TEK Categories Commercialization Analysis:
            cb_uuid appln_id raised_amount_usd                       \
            nunique  nunique               sum          mean  count   
y02_classes                                                           
Y02A           1571     6856      1.126900e+12  3.888276e+07  28982   
Y02B            755     3415      3.246363e+12  3.903286e+08   8317   
Y02C             73      181      1.411143e+10  2.519898e+07    560   
Y02D            946     5527      7.928480e+13  6.301447e+09  12582   
Y02E           1838    12501      2.813509e+12  8.295765e+07  33915   
Y02P           1236     4258      1.373439e+12  1.150573e+08  11937   
Y02T            562     2228      3.682841e+11  7.228343e+07   5095   
Y02W            202      410      3.904737e+10  4.437201e+07    880   

            post_money_valuation_usd investor_count  
                                mean           mean  
y02_classes                                          
Y02A         

In [36]:
# Novelty vs funding success (only if novelty data available)
if 'novelty_q100' in df_tek_funding_novelty.columns:
    novelty_impact = df_tek_funding_novelty.groupby('novelty_category').agg({
        'cb_uuid': 'nunique',
        'raised_amount_usd': ['sum', 'mean', 'count'],
        'post_money_valuation_usd': 'mean'
    }).round(2)
    
    print("Novelty Impact on Funding:")
    print(novelty_impact)

Novelty Impact on Funding:
                 cb_uuid raised_amount_usd                       \
                 nunique               sum          mean  count   
novelty_category                                                  
High                 608      1.248367e+12  1.151099e+08  10845   
Low                  296      1.094402e+12  6.293285e+08   1739   
Medium              2183      1.491639e+13  4.877664e+08  30581   

                 post_money_valuation_usd  
                                     mean  
novelty_category                           
High                         2.354424e+09  
Low                          4.298818e+09  
Medium                       1.530936e+09  


In [37]:
# Novelty vs funding success (only if novelty data available)
if 'novelty_q100' in df_tek_funding_novelty.columns:
    novelty_impact = df_tek_funding_novelty.groupby('novelty_category').agg({
        'cb_uuid': 'nunique',
        'raised_amount_usd': ['sum', 'mean', 'count'],
        'post_money_valuation_usd': 'mean'
    }).round(2)
    
    print("Novelty Impact on Funding:")
    print(novelty_impact)

Novelty Impact on Funding:
                 cb_uuid raised_amount_usd                       \
                 nunique               sum          mean  count   
novelty_category                                                  
High                 608      1.248367e+12  1.151099e+08  10845   
Low                  296      1.094402e+12  6.293285e+08   1739   
Medium              2183      1.491639e+13  4.877664e+08  30581   

                 post_money_valuation_usd  
                                     mean  
novelty_category                           
High                         2.354424e+09  
Low                          4.298818e+09  
Medium                       1.530936e+09  
